In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
%cd /gdrive/MyDrive/nlp-yuan_code/FinBERT-QA
from src.process_data import *

/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Importing packages

In [ ]:
import numpy as np
from sklearn.metrics import ndcg_score
from sklearn.metrics import average_precision_score

Set up the PATH in Google Drive

In [ ]:
RO_FiQA_DATA_PATH = '/gdrive/MyDrive/nlp-data/nlp-qa-datasets/FiQA/FiQA_train_task2/'
RO_FIQA_INDEX = "/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/retriever/lucene-index-fiqa/"
WR_PICKLE_DATA = '/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/'
WR_PICKLE_TRANSIENT_DATA = '/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/transient/'
WR_INTERIM_DATA = '/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/interim/'

Set up the training and test data set

In [ ]:
train_set = load_pickle(WR_INTERIM_DATA + "train_set.pickle")
test_set = load_pickle(WR_INTERIM_DATA + "test_set.pickle")

Calculate the metrics for baseline

In [ ]:
def run_baseline(data):
  ap = [] 
  rr = []
  cg = []

# Data consists of quiestion id, answer labels (in 0 or 1), candidate documents (paraghraphs), and candidate score for ranking answers (from model output)
  for j, seq in enumerate(data):
    qid, ans_labels, cands, cands_score = seq[0], seq[1], seq[2], seq[3]

    rr_ = 0
    ap_ = 0.0
    top_k = 10

    for i in range(len(cands)):
      docid = cands[i]
      if docid in ans_labels and rr_ == 0:
        rr_ = 1/(i+1)
        break
    
    relscores = [1 if docid in ans_labels else 0 for docid in cands]
    pos = [1.0/(i+1) for i in range(len(cands))]
    ap_ = average_precision_score(relscores,pos) if sum(relscores) != 0 else 0 

    mcnt = max(len(cands), len(ans_labels))
    m = dict([(ai, 1/(i+1)) for i, ai in enumerate(ans_labels)])

    if len(ans_labels)==0 or len(cands)==0:
      cg_ = 0.0
    elif len(ans_labels)==1:
      cg_ = 1.0 if len(cands)>0 and cands[0] in ans_labels else 0
    elif len(cands) == 1:
      cg_ = 1.0 if len(ans_labels) and cands[0] in ans_labels else 0
    else:
      relscores = np.asarray([[ m[cands[i]] if i<len(cands) and m.get(cands[i]) else 0 for i in range(mcnt)]])
      pos = np.asarray([[1/(i+1) for i in range(mcnt)]])
      cg_ = ndcg_score(relscores, pos)  

    ap.append(ap_)
    rr.append(rr_)
    cg.append(cg_)
  return rr, ap, cg # returning the metrics MRR, MAP and NDCG

In [ ]:
rr, ap, cg = run_baseline(test_set)

print('Mean Reciprocal Rank (MRR):', np.mean(rr))
print('Mean average Precision (MAP)', np.mean(ap))
print('Normalized Discounted Cumulative Gain (NDCG)', np.mean(cg))

Mean Reciprocal Rank (MRR): 0.30120904839548096
Mean average Precision (MAP) 0.2578318534207535
Normalized Discounted Cumulative Gain (NDCG) 0.2954886911245034
